In [1]:
import torch

In [10]:
t = torch.tensor([1,2,3,4,5]).view(1,1,5).float()
t2 = torch.tensor([2,3,4,5,6]).view(1,1,5).float()

In [11]:
l = [t,t2]

In [13]:
new = torch.stack(l)
print(new)
print(new.size())

tensor([[[[1., 2., 3., 4., 5.]]],


        [[[2., 3., 4., 5., 6.]]]])
torch.Size([2, 1, 1, 5])


In [15]:
new.mean(0).size()

torch.Size([1, 1, 5])